<a href="https://colab.research.google.com/github/tinimini12/Calendar_slots/blob/main/calendar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install icalendar recurring-ical-events pytz requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.2/238.2 kB 12.2 MB/s eta 0:00:00


In [6]:
import requests
from icalendar import Calendar
from datetime import datetime, time, timedelta
import pytz
import recurring_ical_events


def validate_date(prompt):
    """Ask until user enters a valid date."""
    while True:
        value = input(prompt).strip()
        try:
            return datetime.strptime(value, "%Y-%m-%d").date()
        except ValueError:
            print("❌ Invalid date format. Please use YYYY-MM-DD.\n")


def validate_url(prompt):
    """Simple validation: nonempty and starts with http."""
    while True:
        value = input(prompt).strip()
        if value.startswith("http://") or value.startswith("https://"):
            return value
        print("❌ Invalid URL. Must start with http:// or https://\n")


def validate_timezone(prompt):
    """Ask until user enters a valid timezone listed in pytz."""
    while True:
        value = input(prompt).strip()
        if value in pytz.all_timezones:
            return value
        print("❌ Invalid timezone. Example: US/Pacific, UTC, Europe/London\n")


def get_free_slots_outlook(
    start_date,
    end_date,
    ics_url,
    start_hour=9,
    end_hour=17,
    timezone_str="US/Pacific",
    min_free_minutes=30
):
    """Compute free/busy slots within working hours."""

    # Fetch ICS
    response = requests.get(ics_url)
    response.raise_for_status()
    cal = Calendar.from_ical(response.content)

    tz = pytz.timezone(timezone_str)

    free_slots_by_day = {}
    current_day = start_date

    while current_day <= end_date:

        work_start = tz.localize(datetime.combine(current_day, time(start_hour, 0)))
        work_end = tz.localize(datetime.combine(current_day, time(end_hour, 0)))

        events_today = recurring_ical_events.of(cal).between(work_start, work_end)
        busy_events = []

        for event in events_today:
            status = str(event.get("status")).upper() if event.get("status") else "BUSY"
            if status in ["TENTATIVE", "CANCELLED"]:
                continue

            e_start = event.get("dtstart").dt
            e_end = event.get("dtend").dt

            if not isinstance(e_start, datetime):
                continue

            if e_start.tzinfo is None:
                e_start = pytz.UTC.localize(e_start)
            if e_end.tzinfo is None:
                e_end = pytz.UTC.localize(e_end)

            e_start = e_start.astimezone(tz)
            e_end = e_end.astimezone(tz)

            if e_end.date() < current_day or e_start.date() > current_day:
                continue

            e_start_clamped = max(e_start, work_start)
            e_end_clamped = min(e_end, work_end)

            if e_start_clamped < e_end_clamped:
                busy_events.append((e_start_clamped, e_end_clamped))

        busy_events.sort(key=lambda x: x[0])
        merged = []

        for ev in busy_events:
            if not merged:
                merged.append(ev)
            else:
                last_start, last_end = merged[-1]
                if ev[0] <= last_end:
                    merged[-1] = (last_start, max(last_end, ev[1]))
                else:
                    merged.append(ev)

        free_slots = []
        current = work_start
        min_duration = timedelta(minutes=min_free_minutes)

        for start, end in merged:
            if current < start and (start - current) >= min_duration:
                free_slots.append((current, start))
            current = max(current, end)

        if current < work_end and (work_end - current) >= min_duration:
            free_slots.append((current, work_end))

        free_slots_by_day[current_day] = free_slots
        current_day += timedelta(days=1)

    return free_slots_by_day


# ============================================
# MAIN EXECUTION — ONLY RUNS AFTER VALID INPUTS
# ============================================
if __name__ == "__main__":

    print("\n=== FREE TIME FINDER ===\n")

    # --- INPUTS WITH VALIDATION ---
    start_date = validate_date("Enter start date (YYYY-MM-DD): ")
    end_date = validate_date("Enter end date (YYYY-MM-DD): ")

    # Ensure end_date >= start_date
    while end_date < start_date:
        print("❌ End date cannot be earlier than start date.\n")
        end_date = validate_date("Enter end date (YYYY-MM-DD): ")

    ics_url = validate_url("Enter ICS URL: ")
    timezone_str = validate_timezone("Enter timezone (e.g., US/Pacific, UTC): ")

    START_HOUR = 9
    END_HOUR = 17

    print("\nFetching calendar and calculating free slots...\n")

    slots_by_day = get_free_slots_outlook(
        start_date,
        end_date,
        ics_url,
        START_HOUR,
        END_HOUR,
        timezone_str
    )

    # --- OUTPUT RESULTS ---
    for day, slots in slots_by_day.items():
        if slots:
            print(f"\nAvailable Free Slots on {day} ({timezone_str}) ≥30 mins:")
            for start, end in slots:
                print(f"{start.strftime('%Y-%m-%d %I:%M %p')} - "
                      f"{end.strftime('%I:%M %p')}")
        else:
            print(f"\nNo free slots ≥30 minutes on {day}.")



=== FREE TIME FINDER ===

Enter start date (YYYY-MM-DD): 2025-11-21
Enter end date (YYYY-MM-DD): 2025-11-25
Enter ICS URL: https://outlook.office365.com/owa/calendar/23f263c12d6e42979b2f8aa678c26648@earlywarning.com/025cb88f63514be382a032f3295b81294461408820769879715/calendar.ics
Enter timezone (e.g., US/Pacific, UTC): US/Pacific

Fetching calendar and calculating free slots...


Available Free Slots on 2025-11-21 (US/Pacific) ≥30 mins:
2025-11-21 09:00 AM - 10:00 AM
2025-11-21 02:30 PM - 03:05 PM
2025-11-21 03:30 PM - 05:00 PM

Available Free Slots on 2025-11-22 (US/Pacific) ≥30 mins:
2025-11-22 09:00 AM - 10:30 AM
2025-11-22 11:30 AM - 05:00 PM

Available Free Slots on 2025-11-23 (US/Pacific) ≥30 mins:
2025-11-23 09:00 AM - 10:30 AM
2025-11-23 11:30 AM - 05:00 PM

Available Free Slots on 2025-11-24 (US/Pacific) ≥30 mins:
2025-11-24 11:30 AM - 12:30 PM
2025-11-24 03:00 PM - 03:30 PM
2025-11-24 04:00 PM - 05:00 PM

Available Free Slots on 2025-11-25 (US/Pacific) ≥30 mins:
2025-11-25 0